In [ ]:
import random
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from torch.utils.data import Dataset , DataLoader
import cv2

from torchvision import transforms as T,datasets

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.utils import make_grid

from PIL import Image,ImageFilter

import warnings 

import pandas as pd
import requests
import io
import urllib.parse
import pdb
import math

import os
warnings.filterwarnings("ignore")

In [ ]:
!pip install timm # install PyTorch Image Models

In [ ]:
import timm
print(timm.list_models(pretrained=True))

In [ ]:
SOURCE_URL = 'https://storage.googleapis.com/dm-turtle-recall/images.tar'
IMAGE_DIR = '/kaggle/input/turtleimgs/turtle_recall/images'
TAR_PATH = os.path.join(IMAGE_DIR, os.path.basename(SOURCE_URL))
EXPECTED_IMAGE_COUNT = 13891


print(f'The total number of images is: {len(os.listdir(IMAGE_DIR))}')

In [ ]:
BASE_URL = 'https://storage.googleapis.com/dm-turtle-recall/'


def read_csv_from_web(file_name):
  url = urllib.parse.urljoin(BASE_URL, file_name)
  content = requests.get(url).content
  return pd.read_csv(io.StringIO(content.decode('utf-8')))


# Read in csv files.
train = read_csv_from_web('train.csv')
train["type"] = "train"
test = read_csv_from_web('test.csv')
extra = pd.read_csv("../input/extra-turtle-common/extra_turtle_common.csv").sample(n=21, random_state=200).reset_index(drop=True)
extra["type"] = "extra"
new = pd.read_csv("../input/extra-turtle-common/extra_images_new_updated (2).csv").sample(n=21, random_state=200).reset_index(drop=True)
new["type"] = "new"

train_all = pd.concat([train,extra,new])
sample_submission = read_csv_from_web('sample_submission.csv')

In [ ]:
train_all.loc[(train_all.type == 'new'),'turtle_id']='new_turtle'
train_all

In [ ]:
turtle_ids = sorted(np.unique(train_all.turtle_id))
labels_idx = dict(zip(turtle_ids, np.arange(len(turtle_ids))))
label_lookup = {v: k for k, v in labels_idx.items()}
num_classes = len(labels_idx)
assert num_classes == 101
print("number of classes:",num_classes)
image_to_turtle = dict(zip(train_all.image_id, train_all.turtle_id))

image_files = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR)
              if f.split('.')[0] in train_all.image_id.values]

image_ids = [os.path.basename(f).split('.')[0] for f in image_files]
image_turtle_ids = [image_to_turtle[id] for id in image_ids]

In [ ]:
#sanity check
turtle_ids_train = ['new_turtle'] + sorted(np.unique(train.turtle_id))
print(turtle_ids_train == turtle_ids)
assert turtle_ids_train == turtle_ids

In [ ]:
# Load  csv
df_train= train_all.sample(frac=0.9,random_state=200)
df_validation = train_all.drop(df_train.index)

#reset index for train and test 
df_train.reset_index(drop=True, inplace=True)
df_validation.reset_index(drop=True, inplace=True)

# define train paths and targets 
train_image_paths = df_train["image_id"]
train_targets = df_train["turtle_id"]

# define validation paths and targets
val_image_paths = df_validation["image_id"]
val_targets = df_validation["turtle_id"]

#define test paths 
test_image_paths = test["image_id"]

In [ ]:
import seaborn as sns
images_per_turtle = pd.value_counts(df_train['turtle_id'])
print(len(images_per_turtle))
plt.figure(figsize=(3, 21))
sns.barplot(x=images_per_turtle, y=images_per_turtle.index,
            palette='Blues_r', orient='horizontal')
plt.show()

import seaborn as sns
images_per_turtle = pd.value_counts(df_validation['turtle_id'])
print(len(images_per_turtle))
plt.figure(figsize=(3, 21))
sns.barplot(x=images_per_turtle, y=images_per_turtle.index,
            palette='Blues_r', orient='horizontal')
plt.show()

In [ ]:
params = {
    "image_size": 256,
    "model": "resnet152",
    "device": "cuda",
    "lr": 0.001,
    "batch_size": 16,
    "num_workers": 0,
    "epochs": 150
}

In [ ]:
image_size = 256
train_transform = A.Compose(
    [
        A.Resize(params["image_size"],params["image_size"]),
        #A.CenterCrop(224,224),
        A.MedianBlur(blur_limit=5, always_apply=False, p=0.5),
        A.IAASharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), always_apply=False, p=0.5),
        A.HueSaturationValue(hue_shift_limit=0.2,sat_shift_limit=0.2,val_shift_limit=0.2,p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1),contrast_limit=(-0.1, 0.1),p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.7),
        A.Flip(),
        A.FancyPCA(),
        A.GaussNoise(),
        A.IAAEmboss(),
        A.ISONoise(),
        A.CLAHE(clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5),
        A.ColorJitter (brightness=0.5, contrast=0.5, saturation=0.2, hue=0.2, always_apply=False, p=0.5),  
        #A.RandomCrop(height=128, width=128),
        A.RandomRotate90(p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
#         ToTensorV2(),
    ]
)

#Augmentation is not done for test/validation data.
val_transform = A.Compose(
    [
        A.Resize(params["image_size"],params["image_size"]),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
#         ToTensorV2(),
    ]
)

#Augmentation is not done for test/validation data.
test_transform = A.Compose(
    [
        A.Resize(params["image_size"],params["image_size"]),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        ToTensorV2(),
    ]
)

In [ ]:
from PIL import Image, ImageEnhance, ImageOps
import random


class ShearX(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class ShearY(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
            Image.BICUBIC, fillcolor=self.fillcolor)


class TranslateX(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, magnitude * x.size[0] * random.choice([-1, 1]), 0, 1, 0),
            fillcolor=self.fillcolor)


class TranslateY(object):
    def __init__(self, fillcolor=(128, 128, 128)):
        self.fillcolor = fillcolor

    def __call__(self, x, magnitude):
        return x.transform(
            x.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * x.size[1] * random.choice([-1, 1])),
            fillcolor=self.fillcolor)


class Rotate(object):
    def __call__(self, x, magnitude):
        rot = x.convert("RGBA").rotate(magnitude * random.choice([-1, 1]))
        return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(x.mode)


class Color(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Color(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Posterize(object):
    def __call__(self, x, magnitude):
        return ImageOps.posterize(x, magnitude)


class Solarize(object):
    def __call__(self, x, magnitude):
        return ImageOps.solarize(x, magnitude)


class Contrast(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Contrast(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Sharpness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Sharpness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class Brightness(object):
    def __call__(self, x, magnitude):
        return ImageEnhance.Brightness(x).enhance(1 + magnitude * random.choice([-1, 1]))


class AutoContrast(object):
    def __call__(self, x, magnitude):
        return ImageOps.autocontrast(x)


class Equalize(object):
    def __call__(self, x, magnitude):
        return ImageOps.equalize(x)


class Invert(object):
    def __call__(self, x, magnitude):
        return ImageOps.invert(x)

In [ ]:
class ImageNetPolicy(object):
    """ Randomly choose one of the best 24 Sub-policies on ImageNet.
        Example:
        >>> policy = ImageNetPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform = transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     ImageNetPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.4, "posterize", 8, 0.6, "rotate", 9, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "posterize", 7, 0.6, "posterize", 6, fillcolor),
            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),

            SubPolicy(0.4, "equalize", 4, 0.8, "rotate", 8, fillcolor),
            SubPolicy(0.6, "solarize", 3, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.8, "posterize", 5, 1.0, "equalize", 2, fillcolor),
            SubPolicy(0.2, "rotate", 3, 0.6, "solarize", 8, fillcolor),
            SubPolicy(0.6, "equalize", 8, 0.4, "posterize", 6, fillcolor),

            SubPolicy(0.8, "rotate", 8, 0.4, "color", 0, fillcolor),
            SubPolicy(0.4, "rotate", 9, 0.6, "equalize", 2, fillcolor),
            SubPolicy(0.0, "equalize", 7, 0.8, "equalize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),

            SubPolicy(0.8, "rotate", 8, 1.0, "color", 2, fillcolor),
            SubPolicy(0.8, "color", 8, 0.8, "solarize", 7, fillcolor),
            SubPolicy(0.4, "sharpness", 7, 0.6, "invert", 8, fillcolor),
            SubPolicy(0.6, "shearX", 5, 1.0, "equalize", 9, fillcolor),
            SubPolicy(0.4, "color", 0, 0.6, "equalize", 3, fillcolor),

            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor)
        ]

    def __call__(self, image):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](image)

    def __repr__(self):
        return "AutoAugment ImageNet Policy"
    
######################################################################    
    
class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        func = {
            "shearX": ShearX(fillcolor=fillcolor),
            "shearY": ShearY(fillcolor=fillcolor),
            "translateX": TranslateX(fillcolor=fillcolor),
            "translateY": TranslateY(fillcolor=fillcolor),
            "rotate": Rotate(),
            "color": Color(),
            "posterize": Posterize(),
            "solarize": Solarize(),
            "contrast": Contrast(),
            "sharpness": Sharpness(),
            "brightness": Brightness(),
            "autocontrast": AutoContrast(),
            "equalize": Equalize(),
            "invert": Invert()
        }

        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]

    def __call__(self, image):
        if random.random() < self.p1:
            image = self.operation1(image, self.magnitude1)
        if random.random() < self.p2:
            image = self.operation2(image, self.magnitude2)
        return image

In [ ]:
# #define a custom dataset 
from PIL import Image 
import PIL
from torchvision import transforms ,datasets

import tensorflow as tf
class NewCustomDataset(Dataset):
    def __init__(self, image_paths, targets=None, transform=None, train=False):
        self.image_paths=image_paths
        self.targets=targets
        self.transform=transform
        self.train=train
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        
        image_filepath = '/kaggle/input/turtleimgs/turtle_recall/images/'+self.image_paths[idx]+'.JPG'
        #image = cv2.imread(image_filepath)  
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = PIL.Image.open(image_filepath)
       
        if self.targets is not None :
            label = self.targets[idx]
            int_label = labels_idx[label]
            if self.transform is not None :
                if self.train:
                    policy = ImageNetPolicy()
                    aug_img = np.array(policy(image))
                    transform = transforms.ToTensor()
                    image = self.transform(image=aug_img)["image"]
                    image = transform(image)
                else:
                    transform = transforms.ToTensor()
                    image = self.transform(image=np.array(image))["image"]
                    image = transform(image)
            return image ,int_label
        else : 
            label = 0
            if self.transform is not None :
                image = np.array(image)
                image = self.transform(image=image)["image"]
            return image , label
train_dataset  = NewCustomDataset(train_image_paths, train_targets , transform=train_transform, train=True)
val_dataset    = NewCustomDataset(val_image_paths , val_targets , transform=val_transform, train=False)
test_dataset   = NewCustomDataset(test_image_paths, targets=None, transform=test_transform, train=False)

In [ ]:
def visualize_augmentations(dataset, idx=9999, samples=10, cols=5):
    dataset = copy.deepcopy(dataset)
#     dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize, ToTensorV2))])
    rows = samples // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i in range(samples):
        image,_ = dataset[i]
        image = np.transpose(image, (2,1,0)) 
        ax.ravel()[i].imshow(image)
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()
random.seed(52)
# print("validation")
# visualize_augmentations(val_dataset)
print("train")
visualize_augmentations(train_dataset)

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=params["batch_size"], shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)

print("No. of batches in trainloader:{}".format(len(train_loader))) #Trainset Size:  5216 / batch_size: 16 = 326(No. of batches in trainloader) 
print("No. of Total examples:{}".format(len(train_loader.dataset)))

val_loader = DataLoader(
    val_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

print("No. of batches in valloader:{}".format(len(val_loader))) #Trainset Size:  5216 / batch_size: 16 = 326(No. of batches in trainloader) 
print("No. of Total examples:{}".format(len(val_loader.dataset)))

test_loader = DataLoader(
    test_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params["num_workers"], pin_memory=True,
)

print("No. of batches in testloader:{}".format(len(test_loader))) #Trainset Size:  5216 / batch_size: 16 = 326(No. of batches in trainloader) 
print("No. of Total examples:{}".format(len(test_loader.dataset)))

In [ ]:
import timm # PyTorch Image Models
# freeze the first 6 layers of model
model = timm.create_model(params["model"],pretrained=True) #load pretrained model

n_features = model.fc.in_features

ct = 0
for child in model.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

# #we are updating it as a 2-class classifier:
model.fc = nn.Sequential(
    nn.Linear(in_features=n_features, out_features=256), #1792 is the orginal in_features
    nn.ReLU(), #ReLu to be the activation function
    nn.Dropout(p=0.2),
    nn.Linear(in_features=256, out_features=101),
)
model

In [ ]:
class Trainer():
    
    def __init__(self,criterion = None,optimizer = None,schedular = None):
        
        self.criterion = criterion
        self.optimizer = optimizer
        self.schedular = schedular
        self.valid_min_loss = np.Inf
        self.best_epoch = 0

    def accuracy(self,outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))
    
    def train_batch_loop(self,model,trainloader):
        
        train_loss = 0.0
        train_acc = 0.0
        
        for images,labels in tqdm(trainloader): 
            
            images = images.to(device)  
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            train_loss += loss.item()
            train_acc += self.accuracy(logits,labels)
            
        return train_loss / len(trainloader), train_acc / len(trainloader) 

    
    def valid_batch_loop(self,model,validloader):
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for images,labels in tqdm(validloader):
            
            # move the data to CPU
            images = images.to(device) 
            labels = labels.to(device)
            
            logits = model(images)
            loss = self.criterion(logits,labels)
            
            valid_loss += loss.item()
            valid_acc += self.accuracy(logits,labels)
            
            del images
            del labels
            
        return valid_loss / len(validloader), valid_acc / len(validloader)
    
    
    def fit(self,model,trainloader,validloader,epochs):
        
        self.valid_min_loss = np.Inf 
        
        for i in range(epochs):
            
            model.train() # this turn on dropout
            avg_train_loss, avg_train_acc = self.train_batch_loop(model,trainloader) ###
            
            model.eval()  # this turns off the dropout lapyer and batch norm
            avg_valid_loss, avg_valid_acc = self.valid_batch_loop(model,validloader) ###
            
            if avg_valid_loss <= self.valid_min_loss :
                print("Valid_loss decreased {} --> {}".format(self.valid_min_loss,avg_valid_loss))
                torch.save(model.state_dict(),'{}Model_{}_{}.pt'.format(params["model"],i,avg_valid_loss))
                self.valid_min_loss = avg_valid_loss
                self.best_epoch = i
                
            print("Epoch : {} Train Loss : {:.6f} Train Acc : {:.6f}".format(i+1, avg_train_loss, avg_train_acc))
            print("Epoch : {} Valid Loss : {:.6f} Valid Acc : {:.6f}".format(i+1, avg_valid_loss, avg_valid_acc))
    
    def get_valid_min_loss(self):
        return self.valid_min_loss
    
    def get_best_epoch(self):
        return self.best_epoch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("On which device we are on:{}".format(device))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = params["lr"])
model.to(device)
trainer = Trainer(criterion,optimizer)
trainer.fit(model,train_loader,val_loader,epochs = params["epochs"])

In [ ]:
avg_valid_loss = trainer.get_valid_min_loss()
i = trainer.get_best_epoch()

model.load_state_dict(torch.load('./{}Model_{}_{}.pt'.format(params["model"],i,avg_valid_loss)))
model.eval()
trainer = Trainer(criterion,optimizer)

avg_test_loss, avg_test_acc = trainer.valid_batch_loop(model,test_loader)

print("Test Loss : {}".format(avg_test_loss))
print("Test Acc : {}".format(avg_test_acc))

In [ ]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, (img,target) in enumerate(tqdm(data_loader)):
            #input_, target = data['image'], data['target']
            
            output = model(img.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, 5)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

predicts_gpu, confs_gpu, _ = inference(test_loader, model)
predicts, confs = predicts_gpu.cpu().numpy(), confs_gpu.cpu().numpy()

In [ ]:
#labels_idx

key_list = list(labels_idx.keys())
val_list = list(labels_idx.values())

new_list = []
new_sublist = []
row_idx =0 
col_idx = 0
for row in predicts : 
    for val in row : 
        #print(val)
        position = val_list.index(val)
        new_val = key_list[position]
        new_sublist.append(new_val)
    new_list.append(new_sublist)
    new_sublist = []

idx = 0
for sub in new_list:
    sub.insert(0,test["image_id"][idx])
    idx += 1

In [ ]:
df = pd.DataFrame(new_list, columns = ['image_id', 'prediction1','prediction2','prediction3','prediction4','prediction5'])
df.head()

In [ ]:
df.to_csv("submission.csv",index=False)